# Comparison of robustness curves for different models

In [ ]:
import os
os.chdir("../")
import sys
import json
from argparse import Namespace
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import foolbox
from sklearn import metrics
from sklearn.metrics import pairwise_distances as dist
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='paper')

import provable_robustness_max_linear_regions.data as dt
from provable_robustness_max_linear_regions import models
from provable_robustness_max_linear_regions.models import load_model
from robustness_curves import generate_curve_data
from utils import NumpyEncoder

## Plot settings:

In [ ]:
SMALL_SIZE = 4.2
MEDIUM_SIZE = 5.8
BIGGER_SIZE = 6.0

TEXT_WIDTH = 4.8041

TICK_LABEL_TO_TICK_DISTANCE = -2  # the lower the closer

LINE_WIDTH = 0.6


def calc_fig_size(n_rows, n_cols, text_width=TEXT_WIDTH):
    ax_width = text_width / 3
    ax_height = text_width / 5
    extra_height = text_width / 4 * 2 - text_width / 5 * 2

    fig_width = n_cols * ax_width
    fig_height = n_rows * ax_height

    if fig_width > text_width:
        factor = text_width / fig_width
        fig_width *= factor
        fig_height *= factor

    fig_height += extra_height

    return fig_width, fig_height


def tex_rob(sub, sup, arg):
    return 'R_{{{}}}^{{{}}}({{{}}})'.format(sub, sup, arg)


X_EPS = r'perturbation size $\varepsilon$'
X_EPS_INF = r'$\ell_\infty$ perturbation size $\varepsilon$'
X_EPS_ONE = r'$\ell_1$ perturbation size $\varepsilon$'
X_EPS_TWO = r'$\ell_2$ perturbation size $\varepsilon$'
Y_ROB = '${}$'.format(tex_rob('', '', r'\varepsilon'))
Y_ROB_INF = '${}$'.format(tex_rob(r'\|\cdot\|_\infty', '', r'\varepsilon'))
Y_ROB_ONE = '${}$'.format(tex_rob(r'\|\cdot\|_1', '', r'\varepsilon'))
Y_ROB_TWO = '${}$'.format(tex_rob(r'\|\cdot\|_2', '', r'\varepsilon'))

# plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})

plt.rc('font', size=SMALL_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('text', usetex=True)

colors = {
    "orange": sns.xkcd_rgb["yellowish orange"],
    "red": sns.xkcd_rgb["pale red"],
    "green": sns.xkcd_rgb["medium green"],
    "blue": sns.xkcd_rgb["denim blue"],
    "yellow": sns.xkcd_rgb["amber"],
    "purple": sns.xkcd_rgb["dusty purple"],
    "cyan": sns.xkcd_rgb["cyan"]
}

## Calculate robustness curves:
Estimated runtime (if no file with data is present): 10 days

In [ ]:
def load_from_json(file_name):

    if not os.path.exists("res/" + file_name + ".json"):
        return None
    else:
        with open("res/" + file_name + ".json", 'r') as fp:
            loaded_json =  json.load(fp)

            for key in loaded_json.keys():
                loaded_json[key]["x"] = np.array(loaded_json[key]["x"])
                loaded_json[key]["y"] = np.array(loaded_json[key]["y"])

                loaded_json[key]["y"][np.isnan(loaded_json[key]["x"])] = 1.0
                loaded_json[key]["x"] = np.nan_to_num(loaded_json[key]["x"], nan = np.nanmax(loaded_json[key]["x"]))

            return loaded_json
        
def save_to_json(dictionary, file_name):
        
    if not os.path.exists("res"):
        os.makedirs("res")

    with open("res/" + file_name + ".json", 'w') as fp:
        json.dump(dictionary, fp, cls = NumpyEncoder)


training_method_to_model_path = {"ST": "provable_robustness_max_linear_regions/models/plain/2019-02-19 01_20_16 dataset=cifar10 nn_type=cnn_lenet_small p_norm=inf lmbd=0.0 gamma_rb=0.0 gamma_db=0.0 ae_frac=0.0 epoch=100.mat",
                                    "MMR+AT_l_inf": "provable_robustness_max_linear_regions/models/mmr+at/2019-02-17 23_20_04 dataset=cifar10 nn_type=cnn_lenet_small p_norm=inf lmbd=0.1 gamma_rb=3.0 gamma_db=3.0 ae_frac=0.5 epoch=100.mat", 
                                    "MMR+AT_l_2": "provable_robustness_max_linear_regions/models/mmr+at/2019-02-24 10_56_36 dataset=cifar10 nn_type=cnn_lenet_small p_norm=2 lmbd=0.5 gamma_rb=0.15 gamma_db=0.15 ae_frac=0.5 lr=0.001 epoch=100.mat", 
                                    "KW_l_inf": "provable_robustness_max_linear_regions/models/kw/p_norm=inf dataset=cifar10_model=cnn_lenet_small_method=robust_eps=0.007843_checkpoint.mat", 
                                    "KW_l_2": "provable_robustness_max_linear_regions/models/kw/p_norm=2 dataset=cifar10_model=cnn_lenet_small_method=robust_eps=0.1_checkpoint.mat", 
                                    "AT_l_inf": "provable_robustness_max_linear_regions/models/at/2019-02-19 01_20_16 dataset=cifar10 nn_type=cnn_lenet_small p_norm=inf lmbd=0.0 gamma_rb=0.0 gamma_db=0.0 ae_frac=0.5 epoch=100.mat", 
                                    "AT_l_2": "provable_robustness_max_linear_regions/models/at/2019-02-22 02_40_47 dataset=cifar10 nn_type=cnn_lenet_small p_norm=2 lmbd=0.0 gamma_rb=0.0 gamma_db=0.0 ae_frac=0.5 epoch=100.mat",
                                    "MMRUNIV": "experiments/additional_models/mmr_univ_cifar10_gammas_6.0_6.0_lmabdas_1.0_6.0.mat"}

n_points = 10000

_, x_test, _, y_test = dt.get_dataset("cifar10")

x_test = x_test[:n_points]
y_test = y_test[:n_points]

x_test = x_test.reshape(n_points, 1, 32, 32, 3)

model_args = Namespace()
n_test_ex, one, model_args.height, model_args.width, model_args.n_col = x_test.shape
model_args.n_in, model_args.n_out = model_args.height * model_args.width * model_args.n_col, y_test.shape[1]
model_args.n_hs = []
model_args.seed = 1
model_args.nn_type = "cnn"
model_args.dataset = "cifar10"

robustness_curve_data = dict()

for training_method in ["ST", "MMR+AT_l_inf", "MMR+AT_l_2", "KW_l_inf", "KW_l_2", "AT_l_inf", "AT_l_2", "MMRUNIV"]:

    robustness_curve_data[training_method] = load_from_json("rob_curve_data_{}_n_points={}".format(training_method, n_points))

    if not robustness_curve_data[training_method]:
        
        sess = tf.InteractiveSession()
        model, _input, _logits, _ = load_model(sess, model_args, training_method_to_model_path[training_method])
        
        f_model = foolbox.models.TensorFlowModel(_input, _logits, (0,1))
        
        args = Namespace()

        args.inputs = x_test
        args.labels = y_test
        args.f_model = f_model
        args.norms = ["inf", "2"]
        args.save = False
        args.plot = False

        robustness_curve_data[training_method] = generate_curve_data(args)

        save_to_json(robustness_curve_data[training_method], "rob_curve_data_{}_n_points={}".format(training_method, n_points))
        
        tf.reset_default_graph()
        sess.close()

## Plot:

In [ ]:
save_name = "fig_rc_model_comparison_crossings_cifar"

training_method_to_color = {"ST": colors["blue"],
                            "MMR+AT_l_inf": colors["red"], 
                            "MMR+AT_l_2": colors["red"],
                            "MMRUNIV": colors["yellow"],
                            "KW_l_inf": colors["green"], 
                            "KW_l_2": colors["green"], 
                            "AT_l_inf": colors["purple"], 
                            "AT_l_2": colors["purple"]}

training_method_to_col_title = {"ST": "Training: Standard Training",
                            "MMR+AT_l_inf": "Training: MMR+AT\nThreat Model: $\ell_\infty(\epsilon=2/255)$", 
                            "MMR+AT_l_2": "Training: MMR+AT\nThreat Model: $\ell_2(\epsilon=0.1)$",
                            "MMRUNIV": "Training: MMR UNIVERSAL\nThreat Model: all $\ell_p$ norms",
                            "KW_l_inf": "Training: KW\nThreat Model: $\ell_\infty(\epsilon=2/255)$", 
                            "KW_l_2": "Training: KW\nThreat Model: $\ell_2(\epsilon=0.1)$", 
                            "AT_l_inf": "Training: AT\nThreat Model: $\ell_\infty(\epsilon=2/255)$", 
                            "AT_l_2": "Training: AT\nThreat Model: $\ell_2(\epsilon=0.1)$"}



color_map = [colors["blue"], colors["red"], colors["green"]]

# number of model types and parameter combinations
n_cols = 2
n_rows = 1

fig, ax = plt.subplots(n_rows,
                       n_cols,
                       figsize=calc_fig_size(n_rows, n_cols+1))

for training_method in ["ST", "AT_l_inf", "KW_l_inf", "MMR+AT_l_inf", "MMRUNIV"]:
    ax[0].plot(robustness_curve_data[training_method]["inf"]["x"], robustness_curve_data[training_method]["inf"]["y"], c = training_method_to_color[training_method], label = "{}".format(training_method_to_col_title[training_method]), linewidth=LINE_WIDTH)

for training_method in ["ST", "AT_l_2", "KW_l_2", "MMR+AT_l_2", "MMRUNIV"]:
    ax[1].plot(robustness_curve_data[training_method]["2"]["x"], robustness_curve_data[training_method]["2"]["y"], c = training_method_to_color[training_method], label = "{}".format(training_method_to_col_title[training_method]), linewidth=LINE_WIDTH)

ax[0].set_ylabel(Y_ROB_INF)
ax[1].set_ylabel(Y_ROB_TWO)

ax[0].legend(loc="lower right")
ax[1].legend(loc="lower right")

ax[0].set_xlabel(X_EPS_INF)
ax[1].set_xlabel(X_EPS_TWO)

ax[0].set_xlim(right=0.105)

ax[1].set_xlim(right=0.32)

ax[0].set_ylim(-0.04, 1.04)
ax[1].set_ylim(-0.04, 1.04)

ax[0].tick_params(axis='both',
                     which='major',
                     pad=TICK_LABEL_TO_TICK_DISTANCE)
ax[1].tick_params(axis='both',
                     which='major',
                     pad=TICK_LABEL_TO_TICK_DISTANCE)

ax[0]._autoscaleXon = False
ax[0]._autoscaleYon = False
ax[1]._autoscaleXon = False
ax[1]._autoscaleYon = False

ax[0].plot(np.array([1/255, 1/255]), np.array([0.0, 1.0]), c="#000000dd", linewidth=0.3, linestyle='dashed')
ax[0].plot(np.array([4/255, 4/255]), np.array([0.0, 1.0]), c="#000000dd", linewidth=0.3, linestyle='dashed')
ax[0].plot(np.array([8/255, 8/255]), np.array([0.0, 1.0]), c="#000000dd", linewidth=0.3, linestyle='dashed')

ax[0].text(1/255-0.003, -0.06, "$1/255$", fontsize=3)
ax[0].text(4/255-0.003, -0.06, "$4/255$", fontsize=3)
ax[0].text(8/255-0.003, -0.06, "$8/255$", fontsize=3)

fig.tight_layout()
fig.savefig('res/{}.pdf'.format(save_name))